In [ ]:
    async def async_post_stream(
        self,
        path: str,
        headers: Dict[str, str],
        payload: Optional[Dict[str, Any]] = None,
        timeout: Optional[int] = None,
        chunk_size: int = 4096,
    ) -> Optional[aiohttp.ClientResponse]:
        """
        Perform an asynchronous POST request for streaming responses.
        
        Args:
            path: API endpoint path
            headers: HTTP headers
            payload: Request payload
            timeout: Request timeout in seconds
            chunk_size: Size of chunks to read (default: 4096)
            
        Returns:
            Response object that can be used for streaming
        """
        url = self.base_url + path
        client_timeout = aiohttp.ClientTimeout(total=timeout or self.long_timeout)
        
        # For streaming, we need to create a session that will stay open
        # The caller is responsible for closing the session when done
        session = None
        try:
            session = aiohttp.ClientSession(timeout=client_timeout)
            
            response = await session.post(
                url,
                headers=headers,
                json=payload,
                ssl=None if not self.verify_ssl else True,
                raise_for_status=False
            )
            
            # Check for HTTP errors
            if response.status >= 400:
                logger.error(f"Stream request failed: HTTP {response.status}")
                error_body = await response.text()
                logger.error(f"Error response: {error_body}")
                await session.close()
                return None
            
            # Store session and chunk size on the response object for later use
            response._session = session
            response._chunk_size = chunk_size
            return response
                
        except Exception as e:
            logger.error(f"Error in streaming request to {url}: {str(e)}")
            if session:
                await session.close()
            return None